In [2]:
from google.colab import drive
drive.mount('/content/drive')
SEED = 42

from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc
from tqdm import tqdm
import time

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#### MAIN

In [4]:
# -*- coding: utf-8 -*-

'''하기의 코드 및 함수 리스트를 로드 한 후 실행시키는 메인 코드입니다.'''

## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value

## main
def main():
#     _process_data()
    _load_merge_data()
    _prepare_data()
    _build_model()
    _train_predict_amount()
    _train_predict_survival()
    _generate_submission()
    

if __name__ == "__main__":
    
    testname = 'test1'
    print(('*'* 50) +'\ntry model to test1 data\n' + ('*'*50))
    main()
    print('\n\n\nThe end.\n\n\n')
    
    testname = 'test2'
    print(('*'* 50) +'\ntry model to test2 data\n' + ('*'*50))
    main()
    print('\n\n\nThe end.\n\n\n')

**************************************************
try model to test1 data
**************************************************


NameError: ignored

#### ver. 아무튼 이것저것 한거


In [0]:
'''실제 데이터를 load하고 merge하는 곳입니당'''

def _process_data():
    start_vect=time.time()# 시작 시간 저장
    
    codeversion = 'try 9 - 최적화'
    print('CODE VERSION : ', codeversion)
    print('\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

    # load data
    print(' - Loading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')


    ########## merge activity
    print('\n(1) Grouping activity data ...')       
    
    print(' - extract login and logout max days variable')
#     train_all_day = activity_day_data(train_activity)
#     test_all_day = activity_day_data(test_activity)
    
    print(' - create game money change variable')
    for act in train_activity, test_activity : #game_money_change를 마이너스끼리, 플러스끼리 분리한 후 sum
        act['game_money_change_minus'] = act['game_money_change'] 
        act['game_money_change_minus'] = [0 if i > 0 else i for i in act['game_money_change_minus']]
        act['game_money_change'] = [0 if i < 0 else i for i in act['game_money_change']]

    print(' - aggregate variables and merge')
    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}

#     train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True).merge(
#         train_all_day, on = ['acc_id'], how ='left')
#     test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True).merge(
#         test_all_day, on = ['acc_id'], how ='left')

    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    print(' - activity id unique and shape :')
    print('   train  : ', train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print('   test   : ',test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    print('   columns: ', train_activity_merge.columns)
    

    ########## merge payment
    print('\n(2) Grouping payment data ...')
    
    print(' - create payment mean and max variable')
    train_payment_merge = payment_data(train_payment) 
    test_payment_merge = payment_data(test_payment) 

    print(' - payment id unique and shape :')
    print('   train  : ',train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print('   test   : ',test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    print('   columns: ', train_payment_merge.columns)
    
    
    ########## merge combat 
    print('\n(3) Grouping combat data ...')
    
    print(' - create class name dummies')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}

    print(' - aggregate variables and merge')
    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}
    
    for combat in train_combat_merge,test_combat_merge : combat.rename(columns=combat_rename, inplace=True)

    print(' - combat id unique and shape :')
    print('   train  : ',train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print('   test   : ',test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    print('   columns: ', train_combat_merge.columns)
    
    
    ########## merge pledge 
    print('\n(4) Grouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print(' - pledge id unique and shape :')
    print('   train  : ',train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print('   test   : ',test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    print('   columns: ', train_pledge_merge.columns)    
    
    ########## merge trade 
    print('\n(5) Grouping trade data ...')
    
    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server'}
    for trade in train_trade, test_trade :
        trade.rename(columns=trade_rename, inplace=True)

    # trade_type dummies
    print(' - make trade_type dummies')
    type_names = {0:'personal_shop', 1:'trade_shop'}
    for trade in train_trade, test_trade :
        for elem in trade['type'].unique():trade[type_names[elem]] = trade['type'] == elem
            
    # item_type dummies
    print(' - make item_type and trade_time dummies')
    train_trade = trade_dummy(train_trade) 
    test_trade = trade_dummy(test_trade) 
    
    #divide source and target id
    print(' - divide source and target id')
    train_source_trade, train_target_trade = trade_source_target(train_trade)
    test_source_trade, test_target_trade = trade_source_target(test_trade) 
    
    print(' - trade source and target id unique and shape :')
    print('   train  : ',train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print('   test   : ',test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    print('   columns: ', train_source_trade.columns) 
        
    #delete dataframes in memory
    del train_activity,train_combat,train_payment,train_pledge, train_trade
    del test_activity,test_combat,test_payment,test_pledge, test_trade
    
    print("\nSTEP1 is done. training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

    
    ########## merge all data
    print('\nSTEP1-2 : Merging all data ...')
    start_vect=time.time()
            
    print(' - merge train data')
    train_merge = data_merge(train_activity_merge,train_combat_merge,train_pledge_merge,train_payment_merge,train_source_trade,train_target_trade)
    train_merge = train_merge.merge(train_label, on = ['acc_id'], how = 'right')

    print(' - merge test data')
    test_merge = data_merge(test_activity_merge,test_combat_merge,test_pledge_merge,test_payment_merge,test_source_trade,test_target_trade)
        
    print(' - merge id unique and shape :')
    print('   train  : ',train_merge['acc_id'].nunique(), train_merge.shape)
    print('   test   : ',test_merge['acc_id'].nunique(), test_merge.shape)
    print('   train_merge columns: ', train_merge.columns) 
    
    train_merge.to_csv(submission_path + 'merge/' + 'train_merge.csv',index=False)  
    test_merge.to_csv(submission_path + 'merge/' + testname + '_merge.csv',index=False)  
    print('\n - test and train merge data are saved on ' + submission_path + 'merge/' )
    
    print("\nSTEP1-2 is done. training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))
    
    
def _load_merge_data(): 
    
    global train_merge
    global test_merge

    train_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
    test_merge = pd.read_csv(submission_path + 'merge/' + testname + '_merge.csv')
  
    print('\nmerge id unique and shape :')
    print('   train  : ', train_merge['acc_id'].nunique(), train_merge.shape)
    print('   test   : ', test_merge['acc_id'].nunique(), test_merge.shape)
    print('   train_merge columns: ', train_merge.columns) 
    
#     # fill NA
#     train_merge = fill_NA(train_merge)
#     test_merge = fill_NA(test_merge)

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 10, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=2000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=200)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=2000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=200)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*15 #best : 20
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']
    else :
        print('testname is wrong!')
    
    #survival value to integer
    test_predict['survival_time'] = test_predict['survival_time'].astype(int)
    #amount_spent value float round control
    test_predict['amount_spent'] = round(test_predict['amount_spent'], 5)
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)   

#### 함수 리스트

In [0]:
'''data merge를 위한 함수 리스트입니다.'''

def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
#     print('NA ratio: ')
#     print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)  

# def activity_day_data(df) :    
#     acc_id_list = df['acc_id'].unique().tolist()
#     login_day_max, logout_day_max = [], []

#     for i in acc_id_list :
#         dayunique = df[df['acc_id'] == i]['day'].unique().tolist() # 각 user의 접속일 list

#         # (1) login_day_max 변수 생성 
#         day_login,day = [],0
#         for j in range(1, 29) :        
#             if j in dayunique :                  # 각 user의 dayunique에 해당 일자가 있으면 연속으로 day=+1
#                 day += 1
#             else :
#                 day_login.append(day)                    
#                 day = 0                                  # 해당 일자가 dayunique에 없는 경우 day 저장 후 리셋 
#         login_day_max.append(max(day_login))             # 연속 접속일 리스트 중 최대값 append

#         # (2) logout_day 변수 생성
#         day_logout,day = [],0
#         for k in range(1, 29) :
#             if k in dayunique :                 # 각 user가 연속으로 접속하지 않은 경우, day +=1
#                 day += 1
#             else :
#                 day_logout.append(day)
#                 day = 0                                  # 해당 일자에 접속한 경우 day 저장 후 리셋
#         logout_day_max.append(max(day_logout))           # 연속 미접속일 리스트 중 최대값 append

#     all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day'])
#     all_day['acc_id'] = acc_id_list
#     all_day['login_day_max'] = login_day_max
#     all_day['logout_day_max'] = logout_day_max

#     return all_day

def payment_data(df) :
    payment_max = df.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    payment_max.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최대 구매량으로 변경
    
    payment_mean = df.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'mean'}).reset_index(drop=True)
    payment_mean.rename(columns={'amount_spent':'mean_spent'}, inplace=True) #amout_spent를 일일 평균 구매량으로 변경
    payment = payment_max.merge(payment_mean.drop(['day'], axis = 1), on = ['acc_id'], how ='left')

    return payment

def trade_dummy(df) :    
    # item_type dummies    
    df_dummy = pd.get_dummies(df['item_type'])
    # merge dummies
    df = df.drop(columns=['type','item_type'])
    df = pd.concat([df, df_dummy], axis = 1)
    
    # time dummies
    df['time'] = df['time'].astype('str').str[:-6].astype('int')
    df['time'] = df['time'].replace(
        [0,1,2,23], 't23-02').replace([3,4,5,6],'t03-06').replace(
        [7,8,9,10],'t07-10').replace([11,12,13,14],'t11-14').replace(
        [15,16,17,18],'t15-18').replace([19,20,21,22],'t19-22')   
    df_dummy = pd.get_dummies(df['time'])
    # merge dummies
    df = df.drop(columns=['time'])
    df = pd.concat([df, df_dummy], axis = 1)
            
    return df


def trade_source_target(df) :  
    source_trade = df.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    source_trade['source'] = 1
    target_trade = df.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    target_trade['target'] = 1

    #aggregate
    trade_source_agg = {'trade_day':'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        't23-02': 'sum','t03-06': 'sum','t07-10': 'sum','t11-14': 'sum','t15-18': 'sum','t19-22': 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = {'trade_day':'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        't23-02': 'sum','t03-06': 'sum','t07-10': 'sum','t11-14': 'sum','t15-18': 'sum','t19-22': 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    source_trade = source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    target_trade = target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    return source_trade, target_trade




def data_merge(activity,combat,pledge,payment,source,target)  :
    
    # drop the useless columns
    activity = activity.drop(['fishing','revive'], axis = 1)
    combat = combat.drop(['day', 'char_id','server','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class'], axis=1)
    pledge = pledge.drop(['day', 'char_id','server'], axis=1)
    payment = payment.drop(['day'], axis =1)
 
    # merge all!
    df_merge = activity.merge(combat, on=['acc_id'], how='left').merge(
        pledge, on = ['acc_id'], how ='left').merge(
        payment, on = ['acc_id'], how ='left').merge(
        source, on = ['acc_id'], how = 'left').merge(
        target, on = ['acc_id'], how = 'left')

    # create the columns!
    df_merge['attack'] = (df_merge['random_attacker_cnt']+df_merge['combat_random_attacker_cnt']) -(df_merge['random_defender_cnt'] + df_merge['combat_random_defender_cnt'])
    df_merge['cnt'] = df_merge['temp_cnt']+df_merge['same_pledge_cnt']+df_merge['etc_cnt'] + df_merge['combat_temp_cnt']+df_merge['combat_same_pledge_cnt']+df_merge['combat_etc_cnt']
    df_merge['pledge_ratio'] = df_merge['combat_char_cnt']/df_merge['play_char_cnt']
    df_merge['day_spent'] = df_merge['day']*df_merge['mean_spent']
    df_merge['play_spent'] = df_merge['playtime']*df_merge['mean_spent']
    
    drop_merge = ['random_attacker_cnt','random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt','same_pledge_cnt','num_opponent','combat_temp_cnt', 
                  'combat_same_pledge_cnt', 'combat_etc_cnt','combat_random_defender_cnt','combat_random_attacker_cnt','combat_char_cnt','play_char_cnt']
 
    df_merge = df_merge.drop(drop_merge, axis = 1)
    return df_merge 


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### etc

In [3]:
train_activity = pd.read_csv(train_path + 'train_activity.csv')
train_activity.head()

NameError: ignored

In [0]:
def activity_day_data(df) :    
    acc_id_list = df['acc_id'].unique().tolist()
    login_day_max, logout_day_max = [], []

    for i in acc_id_list :
        dayunique = df[df['acc_id'] == i]['day'].unique().tolist() # 각 user의 접속일 list

        # (1) login_day_max 변수 생성 
        day_login,day = [],0
        for j in range(1, 29) :        
            if j in dayunique :                  # 각 user의 dayunique에 해당 일자가 있으면 연속으로 day=+1
                day += 1
            else :
                day_login.append(day)                    
                day = 0                                  # 해당 일자가 dayunique에 없는 경우 day 저장 후 리셋 
        login_day_max.append(max(day_login))             # 연속 접속일 리스트 중 최대값 append

#         # (2) logout_day 변수 생성
#         day_logout,day = [],0
#         for k in range(1, 29) :
#             if k in dayunique :                 # 각 user가 연속으로 접속하지 않은 경우, day +=1
#                 day += 1
#             else :
#                 day_logout.append(day)
#                 day = 0                                  # 해당 일자에 접속한 경우 day 저장 후 리셋
#         logout_day_max.append(max(day_logout))           # 연속 미접속일 리스트 중 최대값 append

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max']) #,'logout_day_max'
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
#     all_day['logout_day_max'] = logout_day_max

    return all_day

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,397380,aa,1.441844,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,1.10514,0.034006,0.0
1,1,75001,216231,aa,0.283219,2.247978,0.047085,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,-0.013963,0.0
2,1,75711,308769,aa,1.036910,2.956638,0.321821,0.166593,0.003470,1,0.245883,0.247337,0.0,0.0,0.00000,-0.001987,0.0
3,1,72230,387177,aa,0.229384,4.042102,0.099147,0.000000,0.001735,0,0.000000,0.000000,0.0,0.0,0.00000,-0.028884,0.0
4,1,34253,339862,aa,1.088405,0.596802,0.003377,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,0.011777,0.0


In [0]:

train_all_day = activity_day_data(train_activity)
train_all_day.head()

# acc_id_list = train_activity['acc_id'].unique().tolist()
# print(acc_id_list)
# login_day_max, logout_day_max = [], []

# dayunique = train_activity[train_activity['acc_id'] == 97731]['day'].unique().tolist()
# print(dayunique)

ValueError: ignored

In [0]:
# print(acc_id_list)

[75001, 75711, 72230, 34253, 83200, 13896, 54399, 67385, 21932, 30937, 53141, 94123, 76110, 128757, 91114, 32521, 20575, 9300, 15556, 69182, 87786, 58956, 3383, 92874, 97731, 121129, 32541, 103844, 3962, 37214, 126805, 25139, 71908, 88787, 69339, 34731, 129153, 108020, 74214, 26317, 81973, 11245, 42080, 110337, 89552, 128586, 112328, 24401, 125051, 52686, 128041, 110777, 123996, 22789, 75145, 23287, 11106, 6525, 87439, 5792, 59116, 14518, 98526, 108745, 26803, 91684, 82317, 33414, 58493, 30769, 70943, 38097, 40010, 97688, 80488, 62071, 74087, 53803, 2746, 112575, 12427, 94483, 61266, 64530, 87709, 12322, 20597, 90585, 115333, 125470, 73660, 9289, 735, 76000, 20524, 8395, 3262, 128601, 98656, 4888, 89332, 122087, 9857, 11169, 78043, 64963, 68468, 36725, 116064, 123633, 93665, 66052, 85275, 59921, 28878, 67763, 20150, 24669, 12866, 67561, 91284, 55442, 13823, 94656, 7871, 22158, 37573, 105788, 2128, 71999, 84709, 62708, 123474, 94565, 13736, 29924, 39184, 102623, 109505, 30650, 85844, 68

In [0]:

# -*- coding: utf-8 -*-


from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    global codeversion
    
    codeversion = 'try 7 - delete and merge variables'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    
    print('CODE VERSION : ', codeversion)
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    acc_id_list = train_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []
    
    print('\n + making day variables for train activity')

    for i in acc_id_list :
        dayunique = train_activity[train_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    train_activity_merge = train_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    print('\n + making day variables for test activity')

    acc_id_list = test_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []

    for i in acc_id_list :
        dayunique = test_activity[test_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    test_activity_merge = test_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
#     train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
#     test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

#     train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
#     test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)



    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'mean'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'mean'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True) #amout_spent를 일일 평균 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    # merge trade 
    print('\nGrouping trade data ...')

    # item_type dummies             
    train_item_type_dummy = pd.get_dummies(train_trade['item_type'])
    test_item_type_dummy = pd.get_dummies(test_trade['item_type'])

    # trade_type dummies
    type_names = {0:'personal_shop', 1:'trade_shop'}
    for elem in train_trade['type'].unique():train_trade[type_names[elem]] = train_trade['type'] == elem
    for elem in test_trade['type'].unique():test_trade[type_names[elem]] = test_trade['type'] == elem

    # merge dummies
    
    train_trade = train_trade.drop(columns=['type','item_type'])
    train_trade = pd.concat([train_trade, train_item_type_dummy], axis = 1)
    
    test_trade = test_trade.drop(columns=['type','item_type'])
    test_trade = pd.concat([test_trade, test_item_type_dummy], axis = 1)

    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server', 'time' : 'trade_time'}
    train_trade.rename(columns=trade_rename, inplace=True)
    test_trade.rename(columns=trade_rename, inplace=True)

    #divide source and target id
    
    train_source_trade = train_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    train_source_trade['source'] = 1
    train_target_trade = train_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    train_target_trade['target'] = 1
    
    test_source_trade = test_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    test_source_trade['source'] = 1
    test_target_trade = test_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    test_target_trade['target'] = 1


    #aggregate
    trade_source_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    train_source_trade = train_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    train_target_trade = train_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    test_source_trade = test_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    test_target_trade = test_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)

    print('trade id unique and shape :')
    print(train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print(test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    train_merge['attack'] = (train_merge['random_attacker_cnt']+train_merge['combat_random_attacker_cnt']) -(train_merge['random_defender_cnt'] + train_merge['combat_random_defender_cnt'])
    test_merge['attack'] = (test_merge['random_attacker_cnt']+test_merge['combat_random_attacker_cnt']) -(test_merge['random_defender_cnt'] + test_merge['combat_random_defender_cnt'])
    
    train_merge['cnt'] = train_merge['temp_cnt']+train_merge['same_pledge_cnt']+train_merge['etc_cnt'] + train_merge['combat_temp_cnt']+train_merge['combat_same_pledge_cnt']+train_merge['combat_etc_cnt']
    test_merge['cnt'] = test_merge['temp_cnt']+test_merge['same_pledge_cnt']+test_merge['etc_cnt'] + test_merge['combat_temp_cnt']+test_merge['combat_same_pledge_cnt']+test_merge['combat_etc_cnt']
    
    
    train_merge['pledge_ratio'] = train_merge['combat_char_cnt']/train_merge['play_char_cnt']
    test_merge['pledge_ratio'] = test_merge['combat_char_cnt']/test_merge['play_char_cnt']
  
    train_merge['day_spent'] = train_merge['day']*train_merge['mean_spent']
    test_merge['day_spent'] = test_merge['day']*test_merge['mean_spent']
    
    drop_merge = ['day','fishing','revive','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class','random_attacker_cnt',
                  'random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt','same_pledge_cnt','num_opponent','combat_temp_cnt', 
                  'combat_same_pledge_cnt', 'combat_etc_cnt','combat_random_defender_cnt','combat_random_attacker_cnt','playtime',
                  'combat_char_cnt','play_char_cnt','mean_spent']
    
    
    
    train_merge = train_merge.drop(drop_merge, axis = 1)
    test_merge = test_merge.drop(drop_merge, axis = 1)
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    train_merge.to_csv(submission_path + '/merge/' + 'train_merge.csv',index=False)  
    test_merge.to_csv(submission_path + '/merge/' + testname + '_merge.csv',index=False)  
    print('test and train merge data are saved on ' + submission_path + '/merge/' )
    
    
def _load_merge_data(): 

    train_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
    test_merge = pd.read_csv(submission_path + 'merge/' + testname + '_merge.csv')
  
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)

    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data



def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 8, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }

    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=5000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=100)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=5000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=20)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*18 #best : 20
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']*5
    else :
        print('testname is wrong!')
        
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    
 

In [0]:
train_label = pd.read_csv(train_path + 'train_label.csv')
train_activity = pd.read_csv(train_path + 'train_activity.csv')
# train_combat = pd.read_csv(train_path + 'train_combat.csv')
# train_payment = pd.read_csv(train_path + 'train_payment.csv')
# train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
# train_trade = pd.read_csv(train_path + 'train_trade.csv')

# test_activity = pd.read_csv(test_path + testname + '_activity.csv')
# test_combat = pd.read_csv(test_path + testname + '_combat.csv')
# test_payment = pd.read_csv(test_path + testname + '_payment.csv')
# test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
# test_trade = pd.read_csv(test_path  + testname + '_trade.csv')


In [0]:
train = train_label.